In [4]:
import pandas as pd

In [10]:
df=pd.read_csv("online_retail_II.csv")

In [12]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,01-12-2009 07:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,01-12-2009 07:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,01-12-2009 07:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,01-12-2009 07:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,01-12-2009 07:45,1.25,13085.0,United Kingdom


In [14]:
df[['InvoiceDate']].head()

,InvoiceDate
0,01-12-2009 07:45
1,01-12-2009 07:45
2,01-12-2009 07:45
3,01-12-2009 07:45
4,01-12-2009 07:45


In [18]:
df=df[~df['Invoice'].astype(str).str.startswith('C')]

In [20]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,01-12-2009 07:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,01-12-2009 07:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,01-12-2009 07:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,01-12-2009 07:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,01-12-2009 07:45,1.25,13085.0,United Kingdom


In [22]:
df = df[df['Quantity'] > 0]

In [24]:
df['TotalSales'] = df['Quantity'] * df['Price']

In [26]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalSales
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,01-12-2009 07:45,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,01-12-2009 07:45,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,01-12-2009 07:45,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,01-12-2009 07:45,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,01-12-2009 07:45,1.25,13085.0,United Kingdom,30.0


In [28]:
customer_summary = df.groupby('Customer ID').agg(
    TotalRevenue=('TotalSales', 'sum'),
    TotalOrders=('Invoice', 'nunique'),
    TotalQuantity=('Quantity', 'sum'),
    LastPurchaseDate=('InvoiceDate', 'max')
).reset_index()

customer_summary.head()

,Customer ID,TotalRevenue,TotalOrders,TotalQuantity,LastPurchaseDate
0,12346.0,77556.46,12,74285,28-06-2010 13:53
1,12347.0,5408.50,7,3094,31-10-2011 12:25
2,12348.0,2019.40,5,2714,27-09-2010 14:59
3,12349.0,4428.69,4,1624,29-04-2010 13:20
4,12350.0,334.40,1,197,02-02-2011 16:01


In [30]:
import datetime as dt
latest_date=df['InvoiceDate'].max()
customer_summary['RecencyDays']=(latest_date-customer_summary['LastPurchaseDate']).dt.days
customer_summary.head()

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [32]:
customer_summary['LastPurchaseDate'] = pd.to_datetime(
    customer_summary['LastPurchaseDate']
)

C:\Users\rohit\AppData\Local\Temp\ipykernel_21660\3804077925.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  customer_summary['LastPurchaseDate'] = pd.to_datetime(


In [34]:
customer_summary['LastPurchaseDate'] = pd.to_datetime(
    customer_summary['LastPurchaseDate'],
    dayfirst=True
)

In [36]:
latest_date = df['InvoiceDate'].max()

customer_summary['RecencyDays'] = (
    latest_date - customer_summary['LastPurchaseDate']
).dt.days

TypeError: unsupported operand type(s) for -: 'DatetimeArray' and 'str'

In [41]:
df['InvoiceDate'] = pd.to_datetime(
    df['InvoiceDate'],
    dayfirst=True
)

In [43]:
latest_date = df['InvoiceDate'].max()

In [45]:
customer_summary['RecencyDays'] = (
    latest_date - customer_summary['LastPurchaseDate']
).dt.days

In [47]:
customer_summary[['Customer ID', 'LastPurchaseDate', 'RecencyDays']].head()

,Customer ID,LastPurchaseDate,RecencyDays
0,12346.0,2010-06-28 13:53:00,523
1,12347.0,2011-10-31 12:25:00,34
2,12348.0,2010-09-27 14:59:00,432
3,12349.0,2010-04-29 13:20:00,583
4,12350.0,2011-02-02 16:01:00,304


In [49]:
frequency = df.groupby('Customer ID')['Invoice'].nunique().reset_index()
frequency.columns = ['Customer ID', 'Frequency']
frequency.head()

,Customer ID,Frequency
0,12346.0,12
1,12347.0,7
2,12348.0,5
3,12349.0,4
4,12350.0,1


In [51]:
customer_summary = customer_summary.merge(
    frequency,
    on='Customer ID',
    how='left'
)

customer_summary.head()

,Customer ID,TotalRevenue,TotalOrders,TotalQuantity,LastPurchaseDate,RecencyDays,Frequency
0,12346.0,77556.46,12,74285,2010-06-28 13:53:00,523,12
1,12347.0,5408.50,7,3094,2011-10-31 12:25:00,34,7
2,12348.0,2019.40,5,2714,2010-09-27 14:59:00,432,5
3,12349.0,4428.69,4,1624,2010-04-29 13:20:00,583,4
4,12350.0,334.40,1,197,2011-02-02 16:01:00,304,1


In [53]:
df['TotalSales'] = df['Quantity'] * df['Price']
df[['Quantity', 'Price', 'TotalSales']].head()

,Quantity,Price,TotalSales
0,12,6.95,83.4
1,12,6.75,81.0
2,12,6.75,81.0
3,48,2.10,100.8
4,24,1.25,30.0


In [55]:
monetary = df.groupby('Customer ID')['TotalSales'].sum().reset_index()
monetary.columns = ['Customer ID', 'MonetaryValue']
monetary.head()

,Customer ID,MonetaryValue
0,12346.0,77556.46
1,12347.0,5408.50
2,12348.0,2019.40
3,12349.0,4428.69
4,12350.0,334.40


In [57]:
customer_summary = customer_summary.merge(
    monetary,
    on='Customer ID',
    how='left'
)

customer_summary.head()

,Customer ID,TotalRevenue,TotalOrders,TotalQuantity,LastPurchaseDate,RecencyDays,Frequency,MonetaryValue
0,12346.0,77556.46,12,74285,2010-06-28 13:53:00,523,12,77556.46
1,12347.0,5408.50,7,3094,2011-10-31 12:25:00,34,7,5408.50
2,12348.0,2019.40,5,2714,2010-09-27 14:59:00,432,5,2019.40
3,12349.0,4428.69,4,1624,2010-04-29 13:20:00,583,4,4428.69
4,12350.0,334.40,1,197,2011-02-02 16:01:00,304,1,334.40


In [59]:
# RFM Scoring (1 = worst, 5 = best)
customer_summary['R_Score'] = pd.qcut(
    customer_summary['RecencyDays'], 
    5, 
    labels=[5,4,3,2,1]
)

customer_summary['F_Score'] = pd.qcut(
    customer_summary['Frequency'], 
    5, 
    labels=[1,2,3,4,5]
)

customer_summary['M_Score'] = pd.qcut(
    customer_summary['MonetaryValue'], 
    5, 
    labels=[1,2,3,4,5]
)

customer_summary.head()

ValueError: Bin edges must be unique: Index([1.0, 1.0, 2.0, 4.0, 8.0, 391.0], dtype='float64', name='Frequency').
You can drop duplicate edges by setting the 'duplicates' kwarg

In [67]:
customer_summary['R_Score'] = pd.qcut(
    customer_summary['RecencyDays'].rank(method='first'), 
    5, 
    labels=[5,4,3,2,1]
)

customer_summary['F_Score'] = pd.qcut(
    customer_summary['Frequency'].rank(method='first'), 
    5, 
    labels=[1,2,3,4,5]
)

customer_summary['M_Score'] = pd.qcut(
    customer_summary['MonetaryValue'].rank(method='first'), 
    5, 
    labels=[1,2,3,4,5]
)

In [69]:
customer_summary.head()

,Customer ID,TotalRevenue,TotalOrders,TotalQuantity,LastPurchaseDate,RecencyDays,Frequency,MonetaryValue,R_Score,F_Score,M_Score
0,12346.0,77556.46,12,74285,2010-06-28 13:53:00,523,12,77556.46,2,5,5
1,12347.0,5408.50,7,3094,2011-10-31 12:25:00,34,7,5408.50,5,4,5
2,12348.0,2019.40,5,2714,2010-09-27 14:59:00,432,5,2019.40,2,4,4
3,12349.0,4428.69,4,1624,2010-04-29 13:20:00,583,4,4428.69,1,3,5
4,12350.0,334.40,1,197,2011-02-02 16:01:00,304,1,334.40,3,1,2


In [71]:
customer_summary['RFM_Score'] = (
    customer_summary['R_Score'].astype(str) +
    customer_summary['F_Score'].astype(str) +
    customer_summary['M_Score'].astype(str)
)

customer_summary[['Customer ID', 'RFM_Score']].head()

,Customer ID,RFM_Score
0,12346.0,255
1,12347.0,545
2,12348.0,244
3,12349.0,135
4,12350.0,312


In [73]:
def rfm_segment(row):
    if row['RFM_Score'] >= '555':
        return 'Champions'
    elif row['RFM_Score'] >= '444':
        return 'Loyal Customers'
    elif row['RFM_Score'] >= '333':
        return 'Potential Loyalists'
    elif row['RFM_Score'] >= '222':
        return 'At Risk'
    else:
        return 'Lost Customers'

customer_summary['Segment'] = customer_summary.apply(rfm_segment, axis=1)

customer_summary[['Customer ID', 'Segment']].head()

,Customer ID,Segment
0,12346.0,At Risk
1,12347.0,Loyal Customers
2,12348.0,At Risk
3,12349.0,Lost Customers
4,12350.0,At Risk


In [75]:
customer_summary['Segment'].value_counts()

Segment
Lost Customers         1491
Loyal Customers        1427
At Risk                1395
Potential Loyalists    1392
Champions               158
Name: count, dtype: int64

In [77]:
customer_summary.head()

,Customer ID,TotalRevenue,TotalOrders,TotalQuantity,LastPurchaseDate,RecencyDays,Frequency,MonetaryValue,R_Score,F_Score,M_Score,RFM_Score,Segment
0,12346.0,77556.46,12,74285,2010-06-28 13:53:00,523,12,77556.46,2,5,5,255,At Risk
1,12347.0,5408.50,7,3094,2011-10-31 12:25:00,34,7,5408.50,5,4,5,545,Loyal Customers
2,12348.0,2019.40,5,2714,2010-09-27 14:59:00,432,5,2019.40,2,4,4,244,At Risk
3,12349.0,4428.69,4,1624,2010-04-29 13:20:00,583,4,4428.69,1,3,5,135,Lost Customers
4,12350.0,334.40,1,197,2011-02-02 16:01:00,304,1,334.40,3,1,2,312,At Risk


In [81]:
customer_summary.to_csv('rfm_customer_summary.csv',index=False)

In [83]:
import os
os.getcwd()

'C:\\Users\\rohit\\Documents'